In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
test=pd.read_csv('C:/Users/yifeichongtian/Desktop/UCL course material/web economics/report/dataset/test.csv')

In [3]:
validation=pd.read_csv('C:/Users/yifeichongtian/Desktop/UCL course material/web economics/report/dataset/validation.csv')

In [4]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299749 entries, 0 to 299748
Data columns (total 26 columns):
click             299749 non-null int64
weekday           299749 non-null int64
hour              299749 non-null int64
bidid             299749 non-null object
logtype           299749 non-null int64
userid            299749 non-null object
useragent         299749 non-null object
IP                299749 non-null object
region            299749 non-null int64
city              299749 non-null int64
adexchange        299749 non-null object
domain            299749 non-null object
url               299749 non-null object
urlid             299749 non-null object
slotid            299749 non-null object
slotwidth         299749 non-null int64
slotheight        299749 non-null int64
slotvisibility    299749 non-null object
slotformat        299749 non-null object
slotprice         299749 non-null int64
creative          299749 non-null object
bidprice          299749 non-null int

In [5]:
validation.advertiser.value_counts()

1458    60025
3386    55196
3427    50381
3476    38839
3358    33853
2821    25632
2259    16419
2261    13370
2997     6034
Name: advertiser, dtype: int64

In [6]:
new_validation=validation['bidprice']
new_validation=pd.DataFrame(validation['bidprice'])
new_validation.columns=['bidprice']

In [7]:
new_validation.bidprice.value_counts()

300    115221
294     39256
238     38786
227     31467
241     25098
277     22199
254     14636
249     13086
Name: bidprice, dtype: int64

In [8]:
validation.bidprice.value_counts()

300    115221
294     39256
238     38786
227     31467
241     25098
277     22199
254     14636
249     13086
Name: bidprice, dtype: int64

In [9]:
validation.adexchange.value_counts()

3       99896
2       92986
1       91118
4        9715
null     6034
Name: adexchange, dtype: int64

In [10]:
validation.loc[validation.adexchange=='3','adexchange':]=3
validation.loc[validation.adexchange=='2','adexchange':]=2
validation.loc[validation.adexchange=='1','adexchange':]=1
validation.loc[validation.adexchange=='4','adexchange':]=4
validation.loc[validation.adexchange=='null','adexchange':]=0

In [11]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [12]:
validation['adexchange'].value_counts()

3    99896
2    92986
1    91118
4     9715
0     6034
Name: adexchange, dtype: int64

In [13]:
validation['userid']=label_encoder.fit_transform(validation['userid'])
validation['useragent']=label_encoder.fit_transform(validation['useragent'])
validation['usertag']=label_encoder.fit_transform(validation['usertag'])
validation['adexchange']=label_encoder.fit_transform(validation['adexchange'])
validation['IP']=label_encoder.fit_transform(validation['IP'])

'hour','weekday','hour','useragent','IP','adexchange','slotwidth','slotheight',

In [14]:
validation['intercept']=1
feature=['click','useragent','usertag','userid','weekday','hour','intercept','adexchange','weekday','hour','city','region']
validation_sample=validation[feature]

In [15]:
validation_sample.corr(method='spearman')

,click,useragent,usertag,userid,weekday,hour,intercept,adexchange,weekday,hour,city,region
click,1.000000,-0.009471,-0.005752,-0.001800,0.001420,0.001790,NaN,-0.005752,0.001420,0.001790,-0.000419,-0.000244
useragent,-0.009471,1.000000,0.043784,0.100422,-0.008815,-0.033304,NaN,0.043784,-0.008815,-0.033304,0.019590,0.017450
usertag,-0.005752,0.043784,1.000000,-0.014228,-0.092711,0.041467,NaN,1.000000,-0.092711,0.041467,-0.039271,-0.039714
userid,-0.001800,0.100422,-0.014228,1.000000,0.000220,-0.020138,NaN,-0.014228,0.000220,-0.020138,-0.057944,-0.057178
weekday,0.001420,-0.008815,-0.092711,0.000220,1.000000,-0.108740,NaN,-0.092711,1.000000,-0.108740,0.011499,0.011219
hour,0.001790,-0.033304,0.041467,-0.020138,-0.108740,1.000000,NaN,0.041467,-0.108740,1.000000,-0.037935,-0.038821
intercept,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
adexchange,-0.005752,0.043784,1.000000,-0.014228,-0.092711,0.041467,NaN,1.000000,-0.092711,0.041467,-0.039271,-0.039714
weekday,0.001420,-0.008815,-0.092711,0.000220,1.000000,-0.108740,NaN,-0.092711,1.000000,-0.108740,0.011499,0.011219
hour,0.001790,-0.033304,0.041467,-0.020138,-0.108740,1.000000,NaN,0.041467,-0.108740,1.000000,-0.037935,-0.038821


In [16]:
x_validation=validation_sample.drop(['click'],axis=1)

In [17]:
validation.click.value_counts()

0    299523
1       226
Name: click, dtype: int64

# Decision tree

In [18]:
decision_tree=DecisionTreeClassifier()
decision_tree.fit(x_validation, validation.click)
decision_tree.score(x_validation, validation.click)
prediction=decision_tree.predict(x_validation)
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()

0    299523
1       226
Name: click, dtype: int64

In [19]:
test.loc[test.adexchange=='3','adexchange':]=3
test.loc[test.adexchange=='2','adexchange':]=2
test.loc[test.adexchange=='1','adexchange':]=1
test.loc[test.adexchange=='4','adexchange':]=4
test.loc[test.adexchange=='null','adexchange':]=0

In [20]:
test['userid']=label_encoder.fit_transform(test['userid'])
test['useragent']=label_encoder.fit_transform(test['useragent'])
test['usertag']=label_encoder.fit_transform(test['usertag'])
test['adexchange']=label_encoder.fit_transform(test['adexchange'])

In [21]:
test['intercept']=1
feature=['useragent','usertag','userid','weekday','hour','intercept','adexchange','weekday','hour','city','region']
test_sample=test[feature]

In [22]:
prediction_test=decision_tree.predict(test_sample)
prediction_test=pd.DataFrame(prediction_test)
prediction_test.columns=['click']
prediction_test.click.value_counts()

0    299415
1       334
Name: click, dtype: int64

In [23]:
Difference=prediction_test.click-prediction.click
Difference=pd.DataFrame(Difference)
Difference.columns=['click']
Difference.click.value_counts()

 0    299189
 1       334
-1       226
Name: click, dtype: int64

In [24]:
Real_accuracy=Difference.click.value_counts()[0]/(Difference.click.value_counts()[1]+Difference.click.value_counts()[-1]+
                                                 Difference.click.value_counts()[0])
Real_accuracy

0.998131770247774

In [25]:
new_file=pd.concat([prediction_test,test],axis=1)
new_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299749 entries, 0 to 299748
Data columns (total 25 columns):
click             299749 non-null int64
weekday           299749 non-null int64
hour              299749 non-null int64
bidid             299749 non-null object
logtype           299749 non-null int64
userid            299749 non-null int64
useragent         299749 non-null int64
IP                299749 non-null object
region            299749 non-null int64
city              299749 non-null int64
adexchange        299749 non-null int64
domain            299749 non-null object
url               299749 non-null object
urlid             299749 non-null object
slotid            299749 non-null object
slotwidth         299749 non-null int64
slotheight        299749 non-null int64
slotvisibility    299749 non-null object
slotformat        299749 non-null object
slotprice         299749 non-null int64
creative          299749 non-null object
keypage           299749 non-null object

## method 1 for decision tree

In [27]:
new_file.click.value_counts()

0    299415
1       334
Name: click, dtype: int64

In [28]:
predicted_CTR=new_file.click.value_counts()[1]/(new_file.click.value_counts()[1]+new_file.click.value_counts()[0])
predicted_CTR

0.0011142656022205245

In [29]:
validation.click.value_counts()

0    299523
1       226
Name: click, dtype: int64

In [30]:
validation_CTR=validation.click.value_counts()[1]/(validation.click.value_counts()[1]+validation.click.value_counts()[0])
validation_CTR

0.00075396415000550463

In [31]:
(new_file.click-validation.click).value_counts()

 0    299198
 1       325
-1       226
Name: click, dtype: int64

In [32]:
validation.advertiser.value_counts()

3    99896
2    92986
1    91118
4     9715
0     6034
Name: advertiser, dtype: int64

In [35]:
validation_two=pd.concat([new_file.click, validation.advertiser,validation.payprice],axis=1)
(validation_two[validation_two.advertiser==1458].click-validation[validation.advertiser==1458].click).value_counts()

 0    59913
 1       62
-1       50
Name: click, dtype: int64

In [36]:
(validation_two[validation_two.advertiser==2997].click-validation[validation.advertiser==2997].click).value_counts()

 0    5999
-1      26
 1       9
Name: click, dtype: int64

In [37]:
(validation_two[validation_two.advertiser==3358].click-validation[validation.advertiser==3358].click).value_counts()

 0    33791
 1       35
-1       27
Name: click, dtype: int64

In [38]:
(validation_two[validation_two.advertiser==3386].click-validation[validation.advertiser==3386].click).value_counts()

 0    55103
 1       60
-1       33
Name: click, dtype: int64

In [39]:
(validation_two[validation_two.advertiser==2261].click-validation[validation.advertiser==2261].click).value_counts()

 0    13348
 1       17
-1        5
Name: click, dtype: int64

In [40]:
sum(validation_two.payprice)

24045218

In [41]:
sum(validation_two[validation_two.click==1].payprice)

24270

In [42]:
sum(validation[validation.advertiser==1458][validation.click==1].payprice)

C:\Users\yifeichongtian\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


4870

In [43]:
validation_two.click.value_counts()

0    299424
1       325
Name: click, dtype: int64

In [56]:
pCTR_1458=validation_two[validation_two.advertiser==1458].click.value_counts()[1]/(validation_two[validation_two.advertiser==1458].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==1458].click.value_counts()[0])
CTR_1458=validation[validation.advertiser==1458].click.value_counts()[1]/(validation[validation.advertiser==1458].click.value_counts()[1]
                                                                          +validation[validation.advertiser==1458].click.value_counts()[0])

bidprice_1458=validation[validation.advertiser==1458].bidprice*pCTR_1458/CTR_1458
bidprice_1458.value_counts()

372.0    60025
Name: bidprice, dtype: int64

In [57]:
pCTR_3386=validation_two[validation_two.advertiser==3386].click.value_counts()[1]/(validation_two[validation_two.advertiser==3386].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==3386].click.value_counts()[0])
CTR_3386=validation[validation.advertiser==3386].click.value_counts()[1]/(validation[validation.advertiser==3386].click.value_counts()[1]
                                                                          +validation[validation.advertiser==3386].click.value_counts()[0])

bidprice_3386=validation[validation.advertiser==3386].bidprice*pCTR_3386/CTR_3386
bidprice_3386.value_counts()

545.454545    55196
Name: bidprice, dtype: int64

In [58]:
pCTR_3427=validation_two[validation_two.advertiser==3427].click.value_counts()[1]/(validation_two[validation_two.advertiser==3427].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==3427].click.value_counts()[0])
CTR_3427=validation[validation.advertiser==3427].click.value_counts()[1]/(validation[validation.advertiser==3427].click.value_counts()[1]
                                                                          +validation[validation.advertiser==3427].click.value_counts()[0])

bidprice_3427=validation[validation.advertiser==3427].bidprice*pCTR_3427/CTR_3427
bidprice_3427.value_counts()

208.866667    19350
206.266667    17454
196.733333    13577
Name: bidprice, dtype: int64

In [59]:
pCTR_3476=validation_two[validation_two.advertiser==3476].click.value_counts()[1]/(validation_two[validation_two.advertiser==3476].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==3476].click.value_counts()[0])
CTR_3476=validation[validation.advertiser==3476].click.value_counts()[1]/(validation[validation.advertiser==3476].click.value_counts()[1]
                                                                          +validation[validation.advertiser==3476].click.value_counts()[0])

bidprice_3476=validation[validation.advertiser==3476].bidprice*pCTR_3476/CTR_3476
bidprice_3476.value_counts()

937.846154    14636
919.384615    13086
878.769231    11117
Name: bidprice, dtype: int64

In [60]:
pCTR_3358=validation_two[validation_two.advertiser==3358].click.value_counts()[1]/(validation_two[validation_two.advertiser==3358].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==3358].click.value_counts()[0])
CTR_3358=validation[validation.advertiser==3358].click.value_counts()[1]/(validation[validation.advertiser==3358].click.value_counts()[1]
                                                                          +validation[validation.advertiser==3358].click.value_counts()[0])

bidprice_3358=validation[validation.advertiser==3358].bidprice*pCTR_3358/CTR_3358
bidprice_3358.value_counts()

294.259259    17890
308.518519    10215
312.407407     5748
Name: bidprice, dtype: int64

In [61]:
pCTR_2821=validation_two[validation_two.advertiser==2821].click.value_counts()[1]/(validation_two[validation_two.advertiser==2821].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==2821].click.value_counts()[0])
CTR_2821=validation[validation.advertiser==2821].click.value_counts()[1]/(validation[validation.advertiser==2821].click.value_counts()[1]
                                                                          +validation[validation.advertiser==2821].click.value_counts()[0])

bidprice_2821=validation[validation.advertiser==2821].bidprice*pCTR_2821/CTR_2821
bidprice_2821.value_counts()

661.50    19986
623.25     5646
Name: bidprice, dtype: int64

In [62]:
pCTR_2259=validation_two[validation_two.advertiser==2259].click.value_counts()[1]/(validation_two[validation_two.advertiser==2259].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==2259].click.value_counts()[0])
CTR_2259=validation[validation.advertiser==2259].click.value_counts()[1]/(validation[validation.advertiser==2259].click.value_counts()[1]
                                                                          +validation[validation.advertiser==2259].click.value_counts()[0])

bidprice_2259=validation[validation.advertiser==2259].bidprice*pCTR_2259/CTR_2259
bidprice_2259.value_counts()

507.818182    10679
478.454545     5740
Name: bidprice, dtype: int64

In [63]:
pCTR_2261=validation_two[validation_two.advertiser==2261].click.value_counts()[1]/(validation_two[validation_two.advertiser==2261].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==2261].click.value_counts()[0])
CTR_2261=validation[validation.advertiser==2261].click.value_counts()[1]/(validation[validation.advertiser==2261].click.value_counts()[1]
                                                                          +validation[validation.advertiser==2261].click.value_counts()[0])

bidprice_2261=validation[validation.advertiser==2261].bidprice*pCTR_2261/CTR_2261
bidprice_2261.value_counts()

999.6    8591
941.8    4779
Name: bidprice, dtype: int64

In [64]:
pCTR_2997=validation_two[validation_two.advertiser==2997].click.value_counts()[1]/(validation_two[validation_two.advertiser==2997].click.value_counts()[1]
                                                                                  +validation_two[validation_two.advertiser==2997].click.value_counts()[0])
CTR_2997=validation[validation.advertiser==2997].click.value_counts()[1]/(validation[validation.advertiser==2997].click.value_counts()[1]
                                                                          +validation[validation.advertiser==2997].click.value_counts()[0])

bidprice_2997=validation[validation.advertiser==2997].bidprice*pCTR_3476/CTR_3476
bidprice_2997.value_counts()

1022.769231    6034
Name: bidprice, dtype: int64

In [78]:
validation_two['bidprice']=1
validation_two[validation_two.advertiser==1458].bidprice==372
validation_two[validation_two.advertiser==3386].bidprice=bidprice_3386
validation_two[validation_two.advertiser==3427].bidprice=bidprice_3427
validation_two[validation_two.advertiser==3476].bidprice=bidprice_3476
validation_two[validation_two.advertiser==3358].bidprice=bidprice_3358
validation_two[validation_two.advertiser==2821].bidprice=bidprice_2821
validation_two[validation_two.advertiser==2259].bidprice=bidprice_2259
validation_two[validation_two.advertiser==2261].bidprice=bidprice_2261
validation_two[validation_two.advertiser==2997].bidprice=bidprice_2997

C:\Users\yifeichongtian\Anaconda3\lib\site-packages\pandas\core\generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [79]:
validation_two.bidprice.value_counts()

1    299749
Name: bidprice, dtype: int64

In [46]:
validation_two.advertiser.value_counts()

1458    60025
3386    55196
3427    50381
3476    38839
3358    33853
2821    25632
2259    16419
2261    13370
2997     6034
Name: advertiser, dtype: int64

## linear bidding strategy

In [32]:
bidprice_test=new_validation.bidprice*((predicted_CTR/validation_CTR))
bidprice_test
print ('max',":",max(bidprice_test))
print ('min',":", min(bidprice_test))

max : 443.362831858
min : 335.477876106


## exponential bidding strategy

In [34]:
bidprice_test=new_validation.bidprice*(np.e**(predicted_CTR/validation_CTR))
bidprice_test
print ('max',":",max(bidprice_test))
print ('min',":", min(bidprice_test))

max : 1315.08762957
min : 995.082973044


## squared bidding strategy

In [31]:
bidprice_test=new_validation.bidprice*((predicted_CTR/validation_CTR)**2)
bidprice_test
print ('max',":",max(bidprice_test))
print ('min',":", min(bidprice_test))

max : 655.235335578
min : 495.794737254


In [47]:
final_test=pd.concat([test.bidid,bidprice_test],axis=1)
final_test
final_test.to_csv('C:/Users/yifeichongtian/Desktop/UCL course material/web economics/report/test_bidprice.csv')

In [32]:
# for validation bidprice
print ('max',":",max(new_validation.bidprice))
print ('min',":", min(new_validation.bidprice))

max : 300
min : 227


In [33]:
# difference for max and min between validation and test bidprice
difference_max=max(bidprice_test)-max(new_validation.bidprice)
difference_min=min(bidprice_test)-min(new_validation.bidprice)
print ('difference for max',":",difference_max)
print ('difference for min',":",difference_min)

difference for max : 316.587046754
difference for min : 239.550865377


In [34]:
new_file=new_file.drop(['advertiser'],axis=1)

In [35]:
validation_price=pd.DataFrame(validation.payprice)
validation_price.columns=['payprice']
final=pd.concat([new_file, bidprice_test,validation_price,validation.advertiser],axis=1)
feature=['bidid','bidprice','click','payprice','advertiser']
final=final[feature]
print (sum(final[final.click==1].payprice))
print (len(final[final.click==1].bidprice))

649
324


In [38]:
final.click.value_counts()

0    299425
1       324
Name: click, dtype: int64

In [ ]:
final

In [ ]:
final.advertiser.value_counts()

# Adaboost

In [80]:
ada_boost=AdaBoostClassifier()
ada_boost.fit(x_validation, validation.click)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [81]:
ada_boost.score(x_validation, validation.click)

0.9992527080991096

In [82]:
prediction=ada_boost.predict(x_validation)
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()

0    299747
1         2
Name: click, dtype: int64

In [ ]:
test.adexchange.value_counts()

In [ ]:
test.loc[test.adexchange=='3','adexchange':]=3
test.loc[test.adexchange=='2','adexchange':]=2
test.loc[test.adexchange=='1','adexchange':]=1
test.loc[test.adexchange=='4','adexchange':]=4
test.loc[test.adexchange=='null','adexchange':]=0

In [ ]:
test['userid']=label_encoder.fit_transform(test['userid'])
test['useragent']=label_encoder.fit_transform(test['useragent'])
test['usertag']=label_encoder.fit_transform(test['usertag'])
test['adexchange']=label_encoder.fit_transform(test['adexchange'])

In [ ]:
test['intercept']=1
feature=['useragent','usertag','userid','weekday','hour','intercept','adexchange','weekday','hour','city','region']
test_sample=test[feature]

In [ ]:
prediction_test=ada_boost.predict(test_sample)
prediction_test=pd.DataFrame(prediction_test)
prediction_test.columns=['click']
prediction_test.click.value_counts()

# KNN

In [ ]:
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(x_validation, validation.click)

In [ ]:
accuracy=knn.score(x_validation, validation.click)
accuracy

In [ ]:
prediction=knn.predict(x_validation)
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()

# SVM

In [ ]:
clf=svm.SVC()
clf.fit(x_validation, validation_1458.click)

In [ ]:
y_validation=validation_1458.click
accuracy=clf.score(x_validation,y_validation)
accuracy

In [ ]:
test_sample=test[test.advertiser==1458]

In [ ]:
test_sample.adexchange.value_counts()

In [ ]:
test[test.advertiser==1458].columns

In [ ]:
test_sample['useragent']=label_encoder.fit_transform(test_sample['useragent'])


In [ ]:
# prediction for click value in testset
test['intercept']=1
feature=['useragent','weekday','hour','intercept']
x_test=test_sample[feature]
prediction=clf.predict(x_test)
prediction

In [ ]:
prediction=pd.DataFrame(prediction)
prediction.columns=['click']
prediction.click.value_counts()